Generate Network Graphs 


for interdiction: 
    * pick random n edge that is not already interdicted


In [29]:
## clear folder contents
import os
import shutil

def clear_folders(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            os.remove(os.path.join(root, file))



In [30]:
import networkx as nx
import matplotlib.pyplot as plt
import math
import numpy
import random
import pynetgen

def random_graph(seed, type, n, m, s, den):
    ## s = n*m for proportionate flow

    """Generate random graph of size (n*n + 2) for given seed and max capacity on any edge and total supply 's'."""
    fileName = "sample.dimacs"

    if type == 'net_graph/':
        ## supply is proportionate 
        # s = n*m
        d = (n*n+2)*den
        # print(f"got calculated density: {d}")
        ## where den = log2(n)
        ## 2*n^2 + 4 == low
        ## n^3+2n == mid

        pynetgen.netgen_generate(seed, nodes=(n*n+2), sinks=1, sources=1, density=d, mincap=1, maxcap=m, rng=1, supply=s, fname=fileName)

    ## grid network 
    elif type == 'grid_graph/':
        d = den
        # pynetgen.grid_generate(seed, rows=n, columns=n, density= d, mincap=1, maxcap=m, rng=1, supply=s, fname=fileName)
        pynetgen.grid_generate(seed, rows=n, columns=n, mincap=1, maxcap=m, rng=1, supply=s, fname=fileName)
       
    ####
    
    # pynetgen.netgen_generate(seed, nodes=n, sinks=1, sources=1, density=4, mincap=0, maxcap=m, supply=s, fname=fileName)

    graph = nx.DiGraph()
    
    ## Puts graph data into a DIMACS file 
    f = open(fileName)
    lines = f.readlines()
    for line in lines:
        if line[0] == 'a':
            tokens = line.split(' ')
            graph.add_edge(int(tokens[1]), int(tokens[2]), capacity=int(tokens[4]))
        
    f.close()

    return graph

# Not the best, but it will create a PNG file that you can at least inspect. Not
# easy to properly draw for everything to be visible and reasable. Quick and Dirty.
def output_graph(G, src, sink, output="output.png"):
    """Output graph to a file. Draw in rectangular layout because that was the graph we constructed."""
    s = int(math.sqrt(sink - 2))
    
    # manual layout since this was set up in an RxC grid
    pos = {}
    for c in range(s):
        for r in range(s):
            pos[2 + r*s + c] = numpy.array([2*c/s, 2*r/s])
    pos[src] = numpy.array([0.5,  -0.5])
    pos[sink] = numpy.array([2.1, 0.5])
    print(pos)
    
    capacity = nx.get_edge_attributes(graph, 'capacity')
    labels = {}
    for u in flow_dict:
        for v in flow_dict[u]:
            labels[(u,v)] = str(int(flow_dict[u][v])) + "/" + str(int(capacity[(u,v)]))
    print(labels)
    
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels, rotate=False)

    options = {                             # Some reasonable defaults
        'linewidths': 2,                    # Node border width
        'node_size': 50,                    # Size of each node
        'width': 2,                         # Edge Width
        'edgecolors':'black',               # Node outlined in black
        'font_size': 10                     # Reasonable font size for labels
    }

    # Draw all edges and nodes
    nx.draw_networkx(graph, pos, **options)

    # Set margins for the axes so that nodes aren't clipped
    plt.axis('off')
    plt.savefig(output)
    plt.clf()

def hypothesis():
    """As the grid layout grows, more supply can go in, but perhaps the ratio of supply/flow approaches a constant."""
    
    # Note the special max_cap,supply constants must also play a role in the final result
    max_cap = 64
    supply = 192
    for n in range(2, 15):
        total = 0
        num_trials = 100
        src = 1
        sink = n*n + 2
        for t in range(num_trials):
            graph = random_graph(n*1000 + t*10, n, max_cap, supply)
            flow_value, flow_dict = nx.maximum_flow(graph, src, sink)
            total += flow_value
            
        average = total/num_trials
        normalized = average / n
        print (n, average, normalized)
    

# import networkx as nx

def write_graph_to_file(graph, source, sink, filepath):
    
    # with open(filename, 'w') as f:
    with open(filepath, 'w') as f:
        f.write(str(source) + '\n')
        f.write(str(sink) + '\n')
        for edge in graph.edges(data=True):
            line = f"{edge[0]}, {edge[1]}, {edge[2]['capacity']}\n"
            f.write(line)

def check_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)


In [31]:
## Iterating for experiments 

def run_experiments(folder, reps, n, m, s, gtype):
    """ Run iterations to generate graphs. \n
        (i) number of graphs per config\n 
        (ii) N nodes, \n
        (iii) m arcs, \n
        (iv) s supply, \n
             """
    ## Density level in the graph: 1-low, 2-mid, 3-high // edges per node?
    # den = 2
    den = 8
    # den = 2


    for i in range(reps):
        ## generate graph with n nodes and m arcs and s supply
        # SEED = random.randint(0, 100) +1
        SEED = i + 1
        print(f"SEED: {SEED}.")
        graph = random_graph(SEED, gtype, n, m, s, den)
        
        source = 1
        sink = graph.number_of_nodes()
        nodeFolder = "nodes_"+str(n)+"/"


        rep_folder = folder + gtype + nodeFolder 
        ## check if folder exists -- create folder
        check_folder(rep_folder )
        


        # output_id = "_"+gtype+"_"+str(SEED)+"_"+str(n)+"_"+str(m)+"_"+str(s)+"_"
        output_id = "density"+str(den)+"_seed"+str(SEED)+"_"
        fileName = rep_folder+ output_id +".txt"
        # 'sample_gr'+str(i)+'_'+output_id+'.txt'


        write_graph_to_file(graph, source, sink, fileName)

        





Graph testing: 

grid and netgen 

graphs -- density and 

1. as n increases -- what is the max flow 
2. interdisction -- random edges for delta max flow 


In [32]:

if __name__ == '__main__':
    # hypothesis()
    
    dir_ = "sample_graphs/"
    ## Clear folders 
    # clear_folders(dir_)

    ## Iterate loop
    SEED = 4
    n = 2
    max_f = 9 
    s = 64
    # s = n*m

    # ## mincap -- set to 1
    # ## number nodes = n^2 for netgen vs nxn grid for gridgen

    maxNodes = 8 ##max number of nodes N in graph
    reps = 10 ## number of replications



    ## Iterate for varying the number of nodes
    for k in range(2, maxNodes+1): 
        n=k 
        m = n*max_f

        run_experiments(dir_, reps, n, m, s, 'net_graph/')
        run_experiments(dir_, reps, n, m, s, 'grid_graph/')


    # # flow_value, flow_dict = nx.maximum_flow(graph, source, sink)
    
    ## Source is below the grid layout.
    ## Sink is to the right of the grid layout.
    # # output_graph(graph, source, sink, 'output'+output_id+'.png')


SEED: 1.
SEED: 2.
SEED: 3.
SEED: 4.
SEED: 5.
SEED: 6.
SEED: 7.
SEED: 8.
SEED: 9.
SEED: 10.
SEED: 1.
SEED: 2.
SEED: 3.
SEED: 4.
SEED: 5.
SEED: 6.
SEED: 7.
SEED: 8.
SEED: 9.
SEED: 10.
SEED: 1.
SEED: 2.
SEED: 3.
SEED: 4.
SEED: 5.
SEED: 6.
SEED: 7.
SEED: 8.
SEED: 9.
SEED: 10.
SEED: 1.
SEED: 2.
SEED: 3.
SEED: 4.
SEED: 5.
SEED: 6.
SEED: 7.
SEED: 8.
SEED: 9.
SEED: 10.
SEED: 1.
SEED: 2.
SEED: 3.
SEED: 4.
SEED: 5.
SEED: 6.
SEED: 7.
SEED: 8.
SEED: 9.
SEED: 10.
SEED: 1.
SEED: 2.
SEED: 3.
SEED: 4.
SEED: 5.
SEED: 6.
SEED: 7.
SEED: 8.
SEED: 9.
SEED: 10.
SEED: 1.
SEED: 2.
SEED: 3.
SEED: 4.
SEED: 5.
SEED: 6.
SEED: 7.
SEED: 8.
SEED: 9.
SEED: 10.
SEED: 1.
SEED: 2.
SEED: 3.
SEED: 4.
SEED: 5.
SEED: 6.
SEED: 7.
SEED: 8.
SEED: 9.
SEED: 10.
SEED: 1.
SEED: 2.
SEED: 3.
SEED: 4.
SEED: 5.
SEED: 6.
SEED: 7.
SEED: 8.
SEED: 9.
SEED: 10.
SEED: 1.
SEED: 2.
SEED: 3.
SEED: 4.
SEED: 5.
SEED: 6.
SEED: 7.
SEED: 8.
SEED: 9.
SEED: 10.
SEED: 1.
SEED: 2.
SEED: 3.
SEED: 4.
SEED: 5.
SEED: 6.
SEED: 7.
SEED: 8.
SEED: 9.
SEED: 10.